In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_6327_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_1844_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_6776_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_5207_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_11703_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_2295_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_9239_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_9010_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_12818_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_6552_label_7.png
/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic/7/id_105

In [6]:
# Import necessary library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to the training and testing datasets
train_path = '/kaggle/input/arabic-characters/Arabic Character Dataset/Train Arabic'
test_path = '/kaggle/input/arabic-characters/Arabic Character Dataset/Test Arabic'
img_size = 224  # Image dimensions (224x224)
batch_size = 32  # Number of images to process in each batch

# Data augmentation and normalization for training and validation data
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values (0-1 range)
    validation_split=0.2  # Use 20% of training data for validation
)

# Generate batches of augmented data for training
train_generator = datagen.flow_from_directory(
    directory=train_path,
    target_size=(img_size, img_size),  # Resize images
    class_mode='categorical',  # Multi-class classification
    batch_size=batch_size,
    subset='training'  # Use subset for training
)

# Generate batches of validation data
val_generator = datagen.flow_from_directory(
    directory=train_path,
    target_size=(img_size, img_size),
    class_mode='categorical',
    batch_size=batch_size,
    subset='validation'  # Use subset for validation
)

# Prepare test data without validation split
datgen = ImageDataGenerator(rescale=1./255)

# Generate batches for testing
test_generator = datgen.flow_from_directory(
    directory=test_path,
    target_size=(img_size, img_size),
    class_mode='categorical',
    batch_size=batch_size
)


Found 10752 images belonging to 28 classes.
Found 2688 images belonging to 28 classes.
Found 3360 images belonging to 28 classes.


In [7]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Path to pre-trained ResNet50 weights (without top layers)
weights_local_path = '/kaggle/input/weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Load ResNet50 model excluding the top layers
resnet_model = ResNet50(weights=weights_local_path, include_top=False)

num_classes = 28  # Number of Arabic alphabet classes

# Freeze pre-trained layers
for layer in resnet_model.layers:
    layer.trainable = False

# Add custom classification layers on top
pool = GlobalAveragePooling2D()(resnet_model.output)
flatten = Flatten()(pool)
predicted_outputs = Dense(num_classes, activation='softmax')(flatten)

# Build the final model
model = Model(inputs=resnet_model.input, outputs=predicted_outputs)


In [8]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Compile the model
model.compile(
    loss='categorical_crossentropy',  # Loss function for multi-class classification
    optimizer=Adam(),  # Adam optimizer
    metrics=['accuracy']  # Track accuracy during training
)

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,  # Stop after 3 epochs with no improvement
    restore_best_weights=True  # Revert to best model weights
)

# Train the model with early stopping# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_generator)

# Print the test accuracy and loss
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")

history = model.fit(
    train_generator,  # Training data
    validation_data=val_generator,  # Validation data
    steps_per_epoch=train_generator.samples // batch_size,  # Number of steps per epoch
    epochs=12,  # Number of epochs to train
    callbacks=[early_stopping]  # Apply early stopping
)


Epoch 1/12
336/336 ━━━━━━━━━━━━━━━━━━━━ 1159s 3s/step - accuracy: 0.1421 - loss: 3.1160 - val_accuracy: 0.3735 - val_loss: 2.4937
Epoch 2/12


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


336/336 ━━━━━━━━━━━━━━━━━━━━ 228s 678ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3735 - val_loss: 2.4937
Epoch 3/12
336/336 ━━━━━━━━━━━━━━━━━━━━ 1164s 3s/step - accuracy: 0.3512 - loss: 2.4253 - val_accuracy: 0.4182 - val_loss: 2.1716
Epoch 4/12
336/336 ━━━━━━━━━━━━━━━━━━━━ 226s 673ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4182 - val_loss: 2.1716
Epoch 5/12
336/336 ━━━━━━━━━━━━━━━━━━━━ 225s 670ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4580 - val_loss: 1.9867
Epoch 7/12
336/336 ━━━━━━━━━━━━━━━━━━━━ 229s 681ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5048 - val_loss: 1.8313
Epoch 9/12
336/336 ━━━━━━━━━━━━━━━━━━━━ 1191s 3s/step - accuracy: 0.5095 - loss: 1.8120 - val_accuracy: 0.5368 - val_loss: 1.7245
Epoch 10/12
336/336 ━━━━━━━━━━━━━━━━━━━━ 229s 682ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5368 - val_loss: 1.7245
Epoch 11/12
336/336 ━━━━━━━━━━━━━━━━━━━━ 1134s 3s/

In [9]:
# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_generator)

# Print the test accuracy and loss
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")


105/105 ━━━━━━━━━━━━━━━━━━━━ 282s 3s/step - accuracy: 0.5238 - loss: 1.6619
Test accuracy : 0.5145833492279053
Test loss : 1.6629420518875122
